In [1]:
import os
import gc
import torch
from torch.utils.data import Dataset
import numpy as np
import random
from transformers import BertModel, BertTokenizer
from torch.utils.data import TensorDataset
from transformers import glue_processors as processors
from transformers import glue_output_modes as output_modes
from transformers import glue_convert_examples_to_features as convert_examples_to_features
import logging

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from transformers import BertForSequenceClassification
from copy import deepcopy
import gc
from sklearn.metrics import accuracy_score
import torch
import numpy as np


In [2]:
class TrainingArgs:
    def __init__(self):
        self.bert_model = 'bert-base-uncased'
        self.local_rank = -1
        self.data_dir = "glue_data"
        self.overwrite_cache = True
        self.num_labels = 2
        self.batch_size = 8
        self.update_lr  = 5e-5
        self.epochs = 3
        self.output_dir = "bert_models&results"
        self.train_sample_per_task = 500
        self.eval_sample_per_task = 100
        self.seed = 2020
args = TrainingArgs()

In [3]:
logger=logging.getLogger()

In [4]:
class BertTask_Baseline(Dataset):
    ''' 
    Before running this script, please makes sure all 8 GLUE datasets are downloaded in local by running python3 ../../utils/download_glue_data.py
    Modified MetaTask takes all 10 GLUE tasks, namely cola, mnli, mnli-mm, mrpc, sst-2, sts-b, 
    qqp, qnli, rte and wnli and convert them from raw test into features. 
    '''
    
    def __init__(self, args, tokenizer, max_seq_length, task, evaluate=False,sample=False):
        """
        :param num_task: number of training tasks.
        :param tokenizer: tokenizer uses to tokenzie from word to sequence
        :param max_seq_length: length of the tokenzier vector
        :param evaluate: indicate whether the dataset is from training/ evaluate sets
        """

        self.tokenizer       = tokenizer
        self.max_seq_length  = max_seq_length
        self.evaluate        = evaluate
        self.local_rank      = args.local_rank
        self.data_dir        = args.data_dir
        self.bert_model      = args.bert_model
        self.overwrite_cache = args.overwrite_cache
        self.sample = sample
        self.task = task
        self.create_batch()
        
        
    def create_batch(self):
        '''
        Randomly select number of examples from each task into supports (meta training dataset) and queries (meta evaluating dataset)
        '''
        
        # 1. randomly select num_task GLUE tasks 
        task = self.task 

        self.dataset = self.load_and_cache_examples(task, self.tokenizer, self.evaluate, self.sample) # map style dataset 


    def load_and_cache_examples(self, task, tokenizer, evaluate=False, sample=False):
        '''
        Copied from official loading and cache scripts from Huggingface Transformer load_and_cache_examples
        https://github.com/huggingface/transformers/blob/master/examples/run_glue.py#L334
        '''
        folder_name = {'cola': 'CoLA', 'mnli-mm':'MNLI'}
        if task in folder_name:
            task_data_path = folder_name[task]
        else:
            task_data_path = task.upper()


        if self.local_rank not in [-1, 0] and not evaluate:
            torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

        processor = processors[task]()
        output_mode = output_modes[task]
        cached_downloaded_file = os.path.join(self.data_dir, task_data_path)
        # print(cached_downloaded_file)

        logger.info(f"Creating features from dataset file at {cached_downloaded_file}")
        label_list = processor.get_labels()

        examples = (
                processor.get_dev_examples(cached_downloaded_file) if evaluate else processor.get_train_examples(cached_downloaded_file)
            )
        if sample:
            if args.seed:
                random.seed(args.seed)
            examples = random.sample(examples, sample)

        features = convert_examples_to_features(
            examples, tokenizer, max_length=self.max_seq_length, label_list=label_list, output_mode=output_mode,
        )

        if self.local_rank == 0 and not evaluate:
            torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

        # Convert to Tensors and build dataset
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
        if output_mode == "classification":
            all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
        elif output_mode == "regression":
            all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

        dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
        return dataset

    def __getitem__(self, index):
        dataset_set = self.dataset[index]
        return dataset_set

    def __len__(self):
        # as we have built up to batchsz of sets, you can sample some small batch size of sets.
        return len(self.dataset)

In [5]:
class Bert_trainer(nn.Module):
    """
    Meta Learner
    """
    def __init__(self, args):
        """
        :param args:
        """
        super(Bert_trainer, self).__init__()

        self.num_labels = args.num_labels
        self.batch_size = args.batch_size
        self.update_lr  = args.update_lr
    

        self.bert_model = args.bert_model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.model = BertForSequenceClassification.from_pretrained(self.bert_model, num_labels = self.num_labels)
        self.outer_optimizer = Adam(self.model.parameters(), lr=self.update_lr)
        self.model.train()

    def forward(self, datasets,training=True):
        """
        batch_tasks = TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, all_label_ids)
        """

        num_task = len(datasets)
        self.model.to(self.device)
        acc = None

        
        if training:
            dataloader = DataLoader(datasets,batch_size=self.batch_size)
            for data in dataloader:
                all_loss = []
                batch = tuple(t.to(self.device) for t in data)
                input_ids, attention_mask, segment_ids, label_id = batch
                outputs = self.model(input_ids, attention_mask, segment_ids, labels = label_id)
                loss = outputs[0]              
                loss.backward()
                self.outer_optimizer.step()
                self.outer_optimizer.zero_grad()
                all_loss.append(loss.item())
            self.model.to(torch.device('cpu'))
            return outputs
        else:
            
            with torch.no_grad():
                correct = 0
                total = 0
                self.model.to(torch.device(self.device))
                dataloader = DataLoader(datasets,batch_size=self.batch_size)
                for data in dataloader:
                    
                    query_batch = iter(dataloader).next()
                    query_batch = tuple(t.to(self.device) for t in query_batch)
                    q_input_ids, q_attention_mask, q_segment_ids, q_label_id = query_batch
                    q_outputs = self.model(q_input_ids, q_attention_mask, q_segment_ids, labels = q_label_id)
                
                    q_logits = F.softmax(q_outputs[1],dim=1)
                    pre_label_id = torch.argmax(q_logits,dim=1)
                    # pre_label_id = pre_label_id.detach().cpu().numpy().tolist()
                    # label_id = q_label_id.detach().cpu().numpy().tolist()
                    total += q_label_id.size(0)
                    correct += pre_label_id.eq(q_label_id.to(self.device).view_as(pre_label_id)).sum().item()
                acc = correct/total
                self.model.to(torch.device('cpu'))
        return acc

In [6]:
random.seed(2020)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
task_lists = ["cola", "sst-2", "mrpc","qqp","qnli","rte"]

my_Bert = Bert_trainer(args)
acc_results = []
saving_path = os.path.join(args.output_dir,"model")
if  not os.path.exists(saving_path):
    os.makedirs(saving_path)

for i,task in enumerate(task_lists):
    train_data = BertTask_Baseline(args, tokenizer,128,task,sample=args.train_sample_per_task)
    print("Training on the {} task".format(task))
    for epoch in range(args.epochs):
        outputs = my_Bert(train_data)
    print("_____***Saving Model***___{}".format(task))
    torch.save(my_Bert.state_dict(), os.path.join(saving_path,"{}_params.pkl".format(task)))
    del train_data
    _ = gc.collect()
    ### Evaluating
    accs = []
    for j in range(i+1):
        eval_task = task_lists[j]
        print("_____Evalating on the {}".format(eval_task))
        eval_data = BertTask_Baseline(args, tokenizer,128,eval_task,evaluate=True,sample=args.eval_sample_per_task)
        acc = my_Bert(eval_data,training=False)
        accs.append(acc)
        del eval_data
        _ = gc.collect()
        print("_____Finishing evalating on the {}".format(eval_task))
    acc_results.append(accs)
    print("Finishing training on the {} task".format(task))


acc_results_pad = [line+[""]*(6-len(line)) for line in acc_results]
final_acc = "\n".join([",".join(list(map(str,line))) for line in acc_results_pad])
    
with open(os.path.join(args.output_dir,"results.txt"),"w") as f:
    f.write(",".join(task_lists))
    f.write("\n")
    f.write(final_acc)


Training on the cola task
_____***Saving Model***___cola
_____Evalating on the cola
_____Finishing evalating on the cola
Finishing training on the cola task
Training on the sst-2 task
_____***Saving Model***___sst-2
_____Evalating on the cola
_____Finishing evalating on the cola
_____Evalating on the sst-2
_____Finishing evalating on the sst-2
Finishing training on the sst-2 task
Training on the mrpc task
_____***Saving Model***___mrpc
_____Evalating on the cola
_____Finishing evalating on the cola
_____Evalating on the sst-2
_____Finishing evalating on the sst-2
_____Evalating on the mrpc
_____Finishing evalating on the mrpc
Finishing training on the mrpc task
Training on the qqp task
_____***Saving Model***___qqp
_____Evalating on the cola
_____Finishing evalating on the cola
_____Evalating on the sst-2
_____Finishing evalating on the sst-2
_____Evalating on the mrpc
_____Finishing evalating on the mrpc
_____Evalating on the qqp
_____Finishing evalating on the qqp
Finishing training 

In [7]:
import pandas as pd
acc_mtx = pd.read_csv(os.path.join(args.output_dir,"results.txt"))
acc_mtx

,cola,sst-2,mrpc,qqp,qnli,rte
0,0.730769,NaN,NaN,NaN,NaN,NaN
1,0.663462,0.961538,NaN,NaN,NaN,NaN
2,0.644231,0.509615,0.298077,NaN,NaN,NaN
3,0.250000,0.250000,0.394231,0.586538,NaN,NaN
4,0.625000,0.750000,0.105769,0.346154,0.682692,NaN
5,0.750000,0.682692,0.615385,0.394231,0.461538,0.653846
